In [1]:
#Imports
import numpy as np
import lmfit as lf
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [2]:
#Setup Blackhole: y = v(r) = G*M*r^-.5

#Fakedata
rdata = np.linspace(0,5,50)                 #Generate r-values
ydata = np.zeros(len(rdata))                #Create array to define later
noise = np.random.normal(0,1,len(rdata))    #Create nosie to add to y-values; otherwise our fits will be to unrealistic data
for i,n in enumerate(rdata):                #Redefine y-data to be:
    ydata[i] = np.sqrt(n*4.3e-6*2.7e9)+noise[i]                #x^2 plus some noise.
sigdata = np.ones(len(rdata))               #Sigma is 1 since that's the width of the noise distribution

#G = 4.300e-6                                #gravitational constant (kpc/solar mass*(km/s)^2)
#Function
def v(r,G,M):          #Independent variable must be listed first for scipy to know how to fit it.
    return np.sqrt(G*M/r)  #In python, anything unset must be listed as a variable in the function, including the parameters we fit for.

#Points representing function - this is necessary to plot f(x) for our comparison.
ycurve = np.sqrt(4.3e-6*2.7e9*rdata)

In [5]:
#Setup
#Initial Guesses for G and M. In this case, our initial guess is the true function.
p0 = [4.3e-6,2.7e9] #Parameters should be in the same order here as they were when we defined our function.

#Do fit
s_fit = opt.curve_fit(v,rdata,ydata,p0,sigma=sigdata,absolute_sigma=True) #absolute_sigma is set so that uncertainties aren't treated as percentages.
#print(s_fit) #If we uncomment this line, we see that s_fit is an array containing two arrays.

#Define parameters from fit. Our parameters are stored in the first array in our fit output, in the order they were listed in f.
s_G = s_fit[0][0]
s_M = s_fit[0][1]
#s_c = s_fit[0][2]

#Define error from fit.
s_cov = s_fit[1]                    #The second array in the fit output is a covariance matrix.
s_error = np.sqrt(np.diag(s_cov)) #The diagonals of the covariance matrix are the variances of individual parameters.
s_Ge = s_error[0]                   #The errors will be in an order corresponding to the order of the parameters in their array
s_Me = s_error[1]
#s_ce = s_error[2]

#Create array to plot
s_curve = s_G*rdata**2+s_M*rdata

#Print Values
print('G: '+str(s_G)+u' \u00B1 '+str(s_Ge))
print('M: '+str(s_M)+u' \u00B1 '+str(s_Me))
#print('c: '+str(s_c)+u' \u00B1 '+str(s_ce))

C:\Users\Cirdan\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 600.

In [ ]:
#Setup
weighdata = 1/sigdata                       #We will need weights for lmfit. This tells us how much to account for a single data point in the fit.
l_mod = lf.Model(v)                         #Tell lmfit that we want to model the function f
params = l_mod.make_params(G=4.3e-6, M=2.7e9)   #Give lmfit our initial guesses - again, the true function

#Do fit
l_fit = l_mod.fit(ydata, params, r=rdata, weights=weighdata) #Here is where the weights we set at the beginning come in.

#Define Stuff
l_dict = l_fit.best_values #l_fit has a lot of output. We want to use the final result.
l_G = l_dict['G']          #Dictionary items are called based on their name.
l_M = l_dict['M']          #So, we don't have to track the order of parameters.
#l_c = l_dict['c']

#Create array to plot
l_curve = l_G*rdata**2+l_M*rdata

l_fit #Display information about the fit

In [ ]:
#Plotting

fig = plt.figure(figsize=(9.0,8.0))                #size of the plot

plt.errorbar(rdata,ydata,yerr=sigdata,fmt='bo',label='Data') #Plot points (fmt='*o') with error bars
plt.plot(rdata,ycurve,label='Function')                      #Accepting default style gives us a solid line
plt.plot(rdata,s_curve,linestyle='--',label='SciPy')         #Plot the scipy curve fit with a dashed line
plt.plot(rdata,l_curve,linestyle='--',label='LmFit')

plt.legend()           #Tells our plot to show a legend
plt.show()             #Depending how your environment is set up, this line may not be necessary, but it won't break anything.